In [1]:
# -*- coding: utf-8 -*
import os
import re
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import warnings
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.backend import tensorflow_backend
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import StratifiedKFold
import scipy.linalg as LA
import cv2
import tensorflow as tf
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras import models, optimizers

/home/tsunei/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth = True))
session = tf.Session(config = config)
tensorflow_backend.set_session(session)

In [3]:
def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

In [4]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_vgg16_trained_model.h5'
 
### データセットの準備
                                                    
image_size = 224
 
X_eva = []
Y_eva = []
    

print("人工物画像のテストデータを取得してます...")
for filepath in list_pictures('../data/CNN_data/test/jinkou/'):
    warnings.filterwarnings('ignore')
    img_eva = img_to_array(load_img(filepath, target_size=(224, 224)))        
    X_eva.append(img_eva)
    Y_eva.append(0) # 教師データ（正解）

print("人工物画像のテストデータを取得しました")

    
# 学習データの取得（非正解画像）
print("自然物画像のテストデータを取得してます...")
for filepath in list_pictures('../data/CNN_data/test/sizen/'):
    img_eva = img_to_array(load_img(filepath, target_size=(224, 224)))
    X_eva.append(img_eva)
    Y_eva.append(1) # 教師データ（正解）

print("自然物画像のテストデータを取得しました")
       
X_eva = np.array(X_eva)   
Y_eva = np.array(Y_eva)  
 
x_test = X_eva.astype('float32')
x_test /= 255

人工物画像のテストデータを取得してます...
人工物画像のテストデータを取得しました
自然物画像のテストデータを取得してます...
自然物画像のテストデータを取得しました


In [14]:
batch_size = 32
num_classes = 2
epochs = 100
ave = 0
fold_num = 3
seed = 5
np.random.seed(seed)
count = 0

kfold = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
jinkou_score = []
sizen_score = []
cvscores = []

for train, test in kfold.split(x_test, Y_eva):
    count = count + 1
    print(count,"/",fold_num)
    ### モデル構築 
    vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
    #vgg_conv.summary()
    for layer in vgg_conv.layers[:-4]:
        layer.trainable = False

    model = models.Sequential()
    model.add(vgg_conv.layers[0])
    model.add(vgg_conv.layers[1])
    model.add(vgg_conv.layers[2])
    model.add(vgg_conv.layers[3])
    model.add(vgg_conv.layers[4])
    model.add(vgg_conv.layers[5])
    model.add(vgg_conv.layers[6])
    model.add(vgg_conv.layers[7])
    model.add(vgg_conv.layers[8])
    model.add(vgg_conv.layers[9])
    model.add(vgg_conv.layers[10])
    model.add(vgg_conv.layers[11])
    model.add(vgg_conv.layers[12])
    model.add(vgg_conv.layers[13])
    model.add(vgg_conv.layers[14])
    model.add(vgg_conv.layers[15])
    model.add(vgg_conv.layers[16])
    model.add(vgg_conv.layers[17])
    model.add(vgg_conv.layers[18])
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='sigmoid'))   ### データは４種類

    #model.summary()

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])

    ### 学習
    model.fit(x_test[train], np_utils.to_categorical(Y_eva[train], 2), batch_size=batch_size, epochs=epochs, validation_data=(x_test[train], np_utils.to_categorical(Y_eva[train], 2)), shuffle=True)

    y_pred = model.predict_classes(x_test[test])

    sum_all_j = 0
    sum_all_s = 0
    sum_j = 0
    sum_s = 0

    for i in range(x_test[test].shape[0]):

        if y_pred[i] == 1:
            if y_pred[i] == Y_eva[test][i]:           
                sum_j = sum_j + 1  

            sum_all_j = sum_all_j + 1  

        else:
            if y_pred[i] == Y_eva[test][i]:
                sum_s = sum_s + 1

            sum_all_s = sum_all_s + 1


    print("人工物画像の正解：",sum_j,"/",sum_all_j,"＝",sum_j / sum_all_j * 100,"%")
    print("自然物画像の正解：",sum_s,"/",sum_all_s,"＝",sum_s / sum_all_s * 100,"%")
    
    sum_j =  sum_j / sum_all_j * 100
    sum_s =  sum_s / sum_all_s * 100
    ave = (sum_s + sum_j) / 2 + ave

    #print("人工物画像の正解率＝",sum_j,"%")
    #print("自然物画像の正解率＝",sum_s,"%")
    #print("全体画像の正解率＝",(sum_s + sum_j) / 2,"%")
      
    # Evaluate
    scores = model.evaluate(x_test[test], keras.utils.to_categorical(Y_eva[test], num_classes), verbose=0)
    #print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    #cvscores.append(scores[1] * 100)
    jinkou_score.append(sum_j)
    sizen_score.append(sum_s)
    cvscores.append((sum_s + sum_j) / 2)
    print(" ")

for i in range(fold_num):
    print((i+1),"/",fold_num)
    print("人工物画像の正解率＝",jinkou_score[i],"%")
    print("自然物画像の正解率＝",sizen_score[i],"%")
    print("全体画像の正解率＝",cvscores[i],"%")

print(" ")
print("平均正解率：%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

1 / 3
Train on 1600 samples, validate on 1600 samples
Epoch 1/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.5388 - acc: 0.7506 - val_loss: 0.3375 - val_acc: 0.9306
Epoch 2/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.2564 - acc: 0.9328 - val_loss: 0.1817 - val_acc: 0.9494
Epoch 3/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.1655 - acc: 0.9453 - val_loss: 0.1404 - val_acc: 0.9584
Epoch 4/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.1400 - acc: 0.9544 - val_loss: 0.1256 - val_acc: 0.9606
Epoch 5/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.1229 - acc: 0.9600 - val_loss: 0.1116 - val_acc: 0.9653
Epoch 6/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.1136 - acc: 0.9603 - val_loss: 0.1028 - val_acc: 0.9669
Epoch 7/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.1054 - acc: 0.9650 - val_loss: 0.0989 - val_acc: 0

1600/1600 [==============================] - 10s 6ms/step - loss: 0.0094 - acc: 0.9997 - val_loss: 0.0073 - val_acc: 1.0000
Epoch 61/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0084 - acc: 0.9997 - val_loss: 0.0069 - val_acc: 1.0000
Epoch 62/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0091 - acc: 0.9994 - val_loss: 0.0068 - val_acc: 1.0000
Epoch 63/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0074 - acc: 1.0000 - val_loss: 0.0065 - val_acc: 1.0000
Epoch 64/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0085 - acc: 0.9994 - val_loss: 0.0066 - val_acc: 1.0000
Epoch 65/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0072 - acc: 1.0000 - val_loss: 0.0061 - val_acc: 1.0000
Epoch 66/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0077 - acc: 0.9997 - val_loss: 0.0056 - val_acc: 1.0000
Epoch 67/100
1600/1600 [==============================

Epoch 19/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0520 - acc: 0.9878 - val_loss: 0.0476 - val_acc: 0.9888
Epoch 20/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0507 - acc: 0.9856 - val_loss: 0.0447 - val_acc: 0.9888
Epoch 21/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0487 - acc: 0.9859 - val_loss: 0.0435 - val_acc: 0.9897
Epoch 22/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0481 - acc: 0.9862 - val_loss: 0.0414 - val_acc: 0.9891
Epoch 23/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0452 - acc: 0.9875 - val_loss: 0.0396 - val_acc: 0.9897
Epoch 24/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0433 - acc: 0.9903 - val_loss: 0.0409 - val_acc: 0.9888
Epoch 25/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0421 - acc: 0.9878 - val_loss: 0.0376 - val_acc: 0.9897
Epoch 26/100
1600/1600 [=================

Epoch 79/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0066 - acc: 0.9994 - val_loss: 0.0048 - val_acc: 1.0000
Epoch 80/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0057 - acc: 0.9997 - val_loss: 0.0046 - val_acc: 1.0000
Epoch 81/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0057 - acc: 0.9997 - val_loss: 0.0045 - val_acc: 1.0000
Epoch 82/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0058 - acc: 0.9997 - val_loss: 0.0044 - val_acc: 1.0000
Epoch 83/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0050 - acc: 1.0000 - val_loss: 0.0042 - val_acc: 1.0000
Epoch 84/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0051 - acc: 1.0000 - val_loss: 0.0042 - val_acc: 1.0000
Epoch 85/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0055 - acc: 0.9997 - val_loss: 0.0042 - val_acc: 1.0000
Epoch 86/100
1600/1600 [=================

Epoch 38/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0193 - acc: 0.9963 - val_loss: 0.0153 - val_acc: 0.9975
Epoch 39/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0166 - acc: 0.9963 - val_loss: 0.0148 - val_acc: 0.9975
Epoch 40/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0170 - acc: 0.9972 - val_loss: 0.0144 - val_acc: 0.9972
Epoch 41/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0173 - acc: 0.9966 - val_loss: 0.0142 - val_acc: 0.9969
Epoch 42/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0149 - acc: 0.9966 - val_loss: 0.0129 - val_acc: 0.9975
Epoch 43/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0152 - acc: 0.9969 - val_loss: 0.0142 - val_acc: 0.9969
Epoch 44/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0138 - acc: 0.9981 - val_loss: 0.0117 - val_acc: 0.9981
Epoch 45/100
1600/1600 [=================

Epoch 98/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0028 - acc: 1.0000 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 99/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0022 - acc: 1.0000 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 100/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0020 - val_acc: 1.0000
人工物画像の正解： 386 / 397 ＝ 97.22921914357683 %
自然物画像の正解： 389 / 403 ＝ 96.52605459057072 %
 
1 / 3
人工物画像の正解率＝ 97.06601466992664 %
自然物画像の正解率＝ 99.23273657289002 %
全体画像の正解率＝ 98.14937562140832 %
2 / 3
人工物画像の正解率＝ 95.37712895377129 %
自然物画像の正解率＝ 97.94344473007712 %
全体画像の正解率＝ 96.6602868419242 %
3 / 3
人工物画像の正解率＝ 97.22921914357683 %
自然物画像の正解率＝ 96.52605459057072 %
全体画像の正解率＝ 96.87763686707378 %
 
平均正解率：97.23% (+/- 0.66%)


In [24]:
### モデル構築 
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
#vgg_conv.summary()
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
print(vgg_conv)
print(vgg_conv.layers[0])
model = models.Sequential()
model.add(vgg_conv.layers[0])
model.add(vgg_conv.layers[1])
model.add(vgg_conv.layers[2])
model.add(vgg_conv.layers[3])
model.add(vgg_conv.layers[4])
model.add(vgg_conv.layers[5])
model.add(vgg_conv.layers[6])
model.add(vgg_conv.layers[7])
model.add(vgg_conv.layers[8])
model.add(vgg_conv.layers[9])
model.add(vgg_conv.layers[10])
model.add(vgg_conv.layers[11])
model.add(vgg_conv.layers[12])
model.add(vgg_conv.layers[13])
model.add(vgg_conv.layers[14])
model.add(vgg_conv.layers[15])
model.add(vgg_conv.layers[16])
model.add(vgg_conv.layers[17])
model.add(vgg_conv.layers[18])
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='sigmoid'))   ### データは４種類
 
model.summary()
 
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
 
### 学習
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    shuffle=True)
 
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Epoch 36/100
800/800 [==============================] - 7s 8ms/step - loss: 0.0396 - acc: 0.9913 - val_loss: 0.1018 - val_acc: 0.9634
Epoch 37/100
800/800 [==============================] - 7s 8ms/step - loss: 0.0397 - acc: 0.9900 - val_loss: 0.1021 - val_acc: 0.9634
Epoch 38/100
800/800 [==============================] - 7s 8ms/step - loss: 0.0391 - acc: 0.9894 - val_loss: 0.1014 - val_acc: 0.9644
Epoch 39/100
800/800 [==============================] - 7s 8ms/step - loss: 0.0368 - acc: 0.9894 - val_loss: 0.1018 - val_acc: 0.9637
Epoch 40/100
800/800 [==============================] - 7s 8ms/step - loss: 0.0346 - acc: 0.9919 - val_loss: 0.1022 - val_acc: 0.9650
Epoch 41/100
800/800 [==============================] - 7s 9ms/step - loss: 0.0371 - acc: 0.9919 - val_loss: 0.1016 - val_acc: 0.9641
Epoch 42/100
800/800 [==============================] - 7s 9ms/step - loss: 0.0345 - acc: 0.9925 - val_loss: 0.1017 - val_acc: 0.9637
Epoch 43/100
800/800 [==============================] - 7s 9ms

In [25]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
 
 
### Plot accuracy &amp; loss
import matplotlib.pyplot as plt 
 
acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
 
epochs = range(1, len(acc) + 1)
 
#plot accuracy
plt.plot(epochs, acc, "bo", label = "Training acc" )
plt.plot(epochs, val_acc, "b", label = "Validation acc")
plt.title("Training and Validation accuracy")
plt.legend()
plt.savefig("acc.png")
plt.close()  

#plot loss
plt.plot(epochs, loss, "bo", label = "Training loss" )
plt.plot(epochs, val_loss, "b", label = "Validation loss")
plt.title("Training and Validation loss")
plt.legend()
plt.savefig("loss.png")
plt.close()

# 正誤表
y_pred = model.predict_classes(x_test)

sum_j = 0
sum_s = 0
for i in range(x_test.shape[0]):
    if i < (x_test.shape[0] / 2):
        if y_pred[i] == 0:
            sum_j = sum_j + 1
            
    else:
        if y_pred[i] == 1:
            sum_s = sum_s + 1
                     
print("人工物画像",sum_j / (x_test.shape[0] / 2) * 100,"%")
print("自然物画像",sum_s / (x_test.shape[0] / 2) * 100,"%")
print("全体画像",(sum_s + sum_j) / x_test.shape[0] * 100,"%")

1600/1600 [==============================] - 4s 3ms/step
Test loss: 0.115434223896591
Test accuracy: 0.9640625
人工物画像 95.375 %
自然物画像 97.5 %
全体画像 96.4375 %
